# 深层神经网络

下面我们直接用minst举例，讲一讲深度神经网络

In [1]:
import numpy as np
import torch as t
from torchvision.datasets import mnist  #从pytorch内置的数据集中导入minst

from torch import nn

In [2]:
#使用内置函数下载mnist数据集
train_set = mnist.MNIST('./data', train = True, download = True)
test_set = mnist.MNIST('./data', train = False, download = True)

In [3]:
#我们可以看看其中一个数据是什么样子的
a_data, a_label = train_set[0]

In [4]:
a_data

In [5]:
a_label

tensor(5)

这里读入的是PIL数据库中的格式，我们可以很方便的将它转换成numpy array

In [6]:
a_data = np.array(a_data, dtype = 'float32')
print(a_data.shape)

(28, 28)


我们可以看到图片的大小是28x28

In [7]:
print(a_data)

[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   3.  18.
   18.  18. 126. 136. 175.  26. 166. 255. 247. 127.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   0.  30.  36.  94. 154. 170. 253.
  253. 253. 253. 253. 225. 172. 253. 242. 195.  64.   0.  

我们可以将数组表示出来，0表示黑色，255表示白色

对于神经网络，第一层的输入就是28x28 = 784，所以需要对数据进行一次变化，用reshape将它们拉平为一维矩阵。

In [8]:
def data_tf(x):
    x = np.array(x, dtype = 'float32') / 255
    x = (x - 0.5) / 0.5
    x = x.reshape((-1, ))
    x = t.from_numpy(x)
    return x

In [9]:
#重新载入数据集，声明定义的数据变换

train_set = mnist.MNIST('./data', train = True, transform = data_tf, download = True)
test_set = mnist.MNIST('./data', train = True, transform = data_tf, download = True)

In [10]:
a, a_label = train_set[0]
print(a.shape)
print(a_label)

torch.Size([784])
tensor(5)


In [11]:
#使用pytorch自带的DataLoader定义一个数据迭代器
from torch.utils.data import DataLoader
train_data = DataLoader(train_set, batch_size = 64, shuffle = True)
test_data = DataLoader(test_set, batch_size = 128, shuffle = False)

使用数据迭代器是很有必要的，如果数据量太大，就无法将它们一次全部读入内存，所以需要使用迭代器，每次传入一部分数据

In [12]:
a, a_label = next(iter(train_data))

In [13]:
#打印出一个批次数据的大小
print(a.shape)
print(a_label.shape)

torch.Size([64, 784])
torch.Size([64])


In [ ]:
#使用Sequential定义一个四层神经网络
net = nn.Sequential(
    nn.Linear(784, 400), 
    nn.ReLU(), 
    nn.Linear(400, 200), 
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.ReLU(),
    nn.Linear(100, 10)
    )

In [ ]:
net

交叉熵在pytorch中已经内置了，交叉熵的数值稳定性更差，内置函数已经帮我们解决了这个问题

In [ ]:
#定义loss函数
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.SGD(net.parameters(), lr = 1e-2)

In [ ]:
#开始训练

losses = []
acces = []
eval_losses = []
eval_acces = []

for e in range(20):
    train_loss = 0
    train_acc = 0
    net.train() #模型设置为训练模式
    for im,label in train_data:
        #前向传播
        out = net(im)
        loss = criterion(out, label)
        
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #记录误差
        train_loss += loss.data.item()
        #计算分类的准确性
        _,pred = out.max(1)
        num_correct = (pred == label).sum().data.item()
        acc = num_correct / im.shape[0]
        train_acc += acc
        
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    
    #在测试集上检验效果
    eval_loss = 0
    eval_acc = 0
    net.eval() #模型设置为测试模式
    for im,label in test_data:
        out = net(im)
        loss = criterion(out, label)
        #记录误差
        eval_loss += loss.data.item()
        #记录准确率
        _, pred = out.max(1)
        num_correct = (pred == label).sum().data.item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
        
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch:{}, Train_loss:{:.6f}, Train_acc:{:.6f}, Eval_loss:{:.6f}, Eval_acc:{:.6f}'.format(e, train_loss/ len(train_data), train_acc/ len(train_data), eval_loss / len(test_data), eval_acc / len(test_data)))

画出loss曲线和准确率曲线

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.title("train_loss")
plt.plot(np.arange(len(losses)), losses)

In [ ]:
plt.title('train_acc')
plt.plot(np.arange(len(acces)), acces)

In [ ]:
plt.title('test_loss')
plt.plot(np.arange(len(eval_losses)), eval_losses)

In [ ]:
plt.title('test_acc')
plt.plot(np.arange(len(eval_acces)), eval_acces)

可以看到我们的三层网络在训练集上能够达到 97.8% 的准确率，测试集上能够达到 98% 的准确率

**小练习：看一看上面的训练过程，看一下准确率是怎么计算出来的，特别注意 max 这个函数**

**自己重新实现一个新的网络，试试改变隐藏层的数目和激活函数，看看有什么新的结果**

准确率的计算方法，我个人的理解是这样的：out是一个行向量，max（1）实际上是按列取最大值，其实还是out本身，只是将label和数据分离开了，后面再通过预测值pred和label值相等的个数（预测正确）和样本总数的比值来作为预测的准确率

In [14]:
#重新实现一个新的网络
net_1 = nn.Sequential(
    nn.Linear(784, 400),
    nn.Tanh(),
    nn.Linear(400,300),
    nn.Tanh(),
    nn.Linear(300, 200),
    nn.Tanh(),
    nn.Linear(200, 100),
    nn.Tanh(),
    nn.Linear(100, 10))

In [15]:
net_1

Sequential(
  (0): Linear(in_features=784, out_features=400, bias=True)
  (1): Tanh()
  (2): Linear(in_features=400, out_features=300, bias=True)
  (3): Tanh()
  (4): Linear(in_features=300, out_features=200, bias=True)
  (5): Tanh()
  (6): Linear(in_features=200, out_features=100, bias=True)
  (7): Tanh()
  (8): Linear(in_features=100, out_features=10, bias=True)
)

In [16]:
#定义loss函数
criterion = nn.CrossEntropyLoss()
optimizer = t.optim.SGD(net_1.parameters(), lr = 1e-2)

In [17]:
#开始训练
losses = []
acces = []
eval_losses = []
eval_acces = []

for e in range(20):
    train_loss = 0
    train_acc = 0
    net_1.train()
    
    for im, label in train_data:
        #前向传播
        out = net_1(im)
        loss = criterion(out, label)
        #反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #记录误差
        train_loss += loss.data.item()
        #计算准确率
        _,pred = out.max(1)
        correct_num = (pred == label).sum().data.item()
        acc = correct_num / im.shape[0]
        train_acc += acc
    losses.append(train_loss / len(train_data))
    acces.append(train_acc / len(train_data))
    
    #在测试集上验证效果
    eval_loss = 0
    eval_acc = 0
    net_1.eval()
    for im, label in test_data:
        out = net_1(im)
        loss = criterion(out, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        eval_loss += loss.data.item()
        
        _, pred = out.max(1)
        num_correct = (pred == label).sum().data.item()
        acc = num_correct / im.shape[0]
        eval_acc += acc
    
    eval_losses.append(eval_loss / len(test_data))
    eval_acces.append(eval_acc / len(test_data))
    print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}, Eval Loss: {:.6f}, Eval Acc: {:.6f}'
          .format(e, train_loss / len(train_data), train_acc / len(train_data), 
                     eval_loss / len(test_data), eval_acc / len(test_data)))

epoch: 0, Train Loss: 1.307801, Train Acc: 0.673874, Eval Loss: 0.492519, Eval Acc: 0.868826
epoch: 1, Train Loss: 0.382268, Train Acc: 0.893573, Eval Loss: 0.325435, Eval Acc: 0.906844
epoch: 2, Train Loss: 0.298539, Train Acc: 0.914062, Eval Loss: 0.265722, Eval Acc: 0.923619
epoch: 3, Train Loss: 0.247401, Train Acc: 0.928871, Eval Loss: 0.219947, Eval Acc: 0.937483
epoch: 4, Train Loss: 0.206424, Train Acc: 0.939666, Eval Loss: 0.182583, Eval Acc: 0.947517
epoch: 5, Train Loss: 0.173956, Train Acc: 0.949544, Eval Loss: 0.153908, Eval Acc: 0.955624
epoch: 6, Train Loss: 0.148721, Train Acc: 0.956890, Eval Loss: 0.133055, Eval Acc: 0.961704
epoch: 7, Train Loss: 0.130514, Train Acc: 0.961887, Eval Loss: 0.116623, Eval Acc: 0.966318
epoch: 8, Train Loss: 0.116745, Train Acc: 0.965902, Eval Loss: 0.103472, Eval Acc: 0.970133
epoch: 9, Train Loss: 0.105424, Train Acc: 0.968733, Eval Loss: 0.092620, Eval Acc: 0.973270
epoch: 10, Train Loss: 0.095491, Train Acc: 0.971932, Eval Loss: 0.083